In [1]:
import numpy as np


In [2]:

def chunks(l,n):
  """Yield n-sized chunks from l"""
  for i in xrange(0, len(l), n):
    yield l[i:i+n]


def get_neighbours(surfname):
  """Get neighbours from obj file"""
  Polys=[]
  k=0
  with open(surfname,'r') as fp:
    for i, line in enumerate(fp):
      if i==0:
    #Number of vertices
        n_vert=int(line.split()[6])
      elif i==2*n_vert+3:
        n_poly=int(line)
      elif i>2*n_vert+5:
        if not line.strip():
          k=1
        elif k==1:
          Polys.extend(line.split())
  Polys=map(int, Polys)
  tris=list(chunks(Polys,3))
  neighbours=[[] for i in range(n_vert)]
  for tri in tris:
      neighbours[tri[0]].extend([tri[1],tri[2]])
      neighbours[tri[2]].extend([tri[1],tri[0]])
      neighbours[tri[1]].extend([tri[0],tri[2]])
#Get unique neighbours
  for k in range(len(neighbours)):
      neighbours[k]=(list(set(neighbours[k])))
  return neighbours;


In [16]:
hemi='left'
Indices=np.loadtxt('/data1/data/kwagstyl/bigbrain/cortical_layers/'+hemi+'_indices.txt',dtype=int)
neighbours=get_neighbours('/data1/data/kwagstyl/bigbrain/TestData/white_'+hemi+'_up.obj')

In [17]:
ZeroRows=np.where(~Indices.any(axis=1))[0]

In [18]:
NewIndices=np.zeros((len(ZeroRows),7))
k=-1
for z in ZeroRows:
    k+=1
    N1=neighbours[z]
    N2=[]
    for N in N1:
        N2.extend(neighbours[N])
    N2=np.unique(N2)
    Nonzeros=np.where(Indices.any(axis=1))[0]
    if not Nonzeros.any():
        N3=[]
        for N in N2:
            N3.extend(neighbours[N])
        N2=np.unique(N3)
    Nonzeros=np.where(Indices[N2].any(axis=1))[0]
    if Nonzeros.any():
        I=np.round(np.mean(Indices[Nonzeros],axis=0))
        NewIndices[k]=I

In [19]:
Indices[ZeroRows]=NewIndices

In [20]:
np.savetxt('/data1/data/kwagstyl/bigbrain/TestData/'+hemi+'_indices_nonzeros.txt',Indices,fmt='%i')

In [75]:
np.shape(Indices)

(655362, 7)

In [93]:
np.shape(NewIndices)

(1049, 7)

In [87]:
Indices[ZeroRows]

array([[ 31,  55,  73, ..., 125, 151, 180],
       [ 30,  50,  69, ..., 128, 153, 181],
       [ 46,  67,  84, ..., 126, 152, 176],
       ..., 
       [ 31,  56,  75, ..., 128, 152, 182],
       [ 31,  53,  73, ..., 126, 152, 181],
       [ 29,  52,  71, ..., 125, 151, 180]])

In [96]:
np.where(Indices<0)

(array([], dtype=int64), array([], dtype=int64))